# Tutorial 3: Write Your Own GNN Module
Source: https://docs.dgl.ai/tutorials/blitz/3_message_passing.html

In [2]:
# Import 
import os
os.environ['DGLBACKEND'] = 'pytorch'
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn

## Message Passing and GNNs

Message passing framework:

$m_{u \rightarrow v}^{(l)} = M^{(l)}(h_v^{(l-1)}, h_u^{(l-1)}, e_{u \rightarrow v}^{(l-1)})$

$m_v^{(l)} = \sum_{u \in \mathcal{N}(v)} m_{u \to v}^{(l)}$

$h_v^{(l)} = U^{l} ( h_v^{(l-1)}, m_v^{(l)} )$

Where $M^{(l)}$ is the message function, $\sum$ is the reduce function (not necessarily a summation) and $U^{(l)}$ is the update function.

In this tutorial we create the $\sum$ reduce function to be the GraphSAGE Convolution as follows:

$h_{\mathcal{N}(v)}^k \leftarrow Average\{h_{u}^{k-1}, \forall u \in \mathcal{N}(v)\}$

$h_v^k \leftarrow ReLU (W^k \cdot CONCAT(h_v^{k-1}, h_{\mathcal{N}(v)}^k)$

In [16]:
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule to transform the input and neighbor features to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
        
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Gathers and averages the neighborhood features. Update all triggers this for all nodes and edges.
            g.update_all(
                # Copies the node feature under name 'h' as messages under name 'm' which is sent to neighbors.
                message_func=fn.copy_u("h", "m"),
                # Averages all the received messages under name 'm' and stores the result under name 'h_N'.
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [17]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
    

## Training Loop

In [18]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [23]:

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0
    
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward pass
        logits = model(g, features)
        
        # Compute prediction
        pred = logits.argmax(1)
        
        # Compute loss
        # Only computes the loss for the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # Compute train/val/test accuracy
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # Save the best validation accuracy and the corresponding test accuracy
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())
        
        if e % 5 == 0:
            print(f"In epoch {e}, loss: {loss:.4f}, val acc: {val_acc:.4f} (Best:{best_val_acc:.4f}), test acc: {test_acc:.4f} (Best:{best_test_acc:.4f})")
        

In [24]:
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.9536, val acc: 0.1220 (Best:0.1220), test acc: 0.1310 (Best:0.1310)
In epoch 5, loss: 1.8741, val acc: 0.2100 (Best:0.2100), test acc: 0.2290 (Best:0.2290)
In epoch 10, loss: 1.7257, val acc: 0.3740 (Best:0.3740), test acc: 0.3730 (Best:0.3730)
In epoch 15, loss: 1.5059, val acc: 0.4040 (Best:0.4040), test acc: 0.4020 (Best:0.4020)
In epoch 20, loss: 1.2276, val acc: 0.5100 (Best:0.5100), test acc: 0.4850 (Best:0.4850)
In epoch 25, loss: 0.9225, val acc: 0.6260 (Best:0.6260), test acc: 0.5880 (Best:0.5880)
In epoch 30, loss: 0.6331, val acc: 0.7120 (Best:0.7120), test acc: 0.6840 (Best:0.6840)
In epoch 35, loss: 0.3999, val acc: 0.7440 (Best:0.7440), test acc: 0.7450 (Best:0.7450)
In epoch 40, loss: 0.2390, val acc: 0.7560 (Best:0.7560), test acc: 0.7540 (Best:0.7540)
In epoch 45, loss: 0.1398, val acc: 0.7580 (Best:0.7580), test acc: 0.7590 (Best:0.7590)
In epoch 50, loss: 0.0832, val acc: 0.7640 (Best:0.7640), test acc: 0.7560 (Best:0.7560)
In epoch 55, loss: 0.05

In [25]:
class WeightedSAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        self.linear = nn.Linear(in_feat * 2, out_feat)
        
    def forward(self, g, h, w):
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [28]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(),1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(),1).to(g.device))
        return h

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.9566, val acc: 0.0680 (Best:0.0680), test acc: 0.0990 (Best:0.0990)
In epoch 5, loss: 1.8758, val acc: 0.1640 (Best:0.1640), test acc: 0.1900 (Best:0.1900)
In epoch 10, loss: 1.7223, val acc: 0.4900 (Best:0.4900), test acc: 0.4800 (Best:0.4800)
In epoch 15, loss: 1.4957, val acc: 0.4980 (Best:0.4980), test acc: 0.4970 (Best:0.4970)
In epoch 20, loss: 1.2033, val acc: 0.6080 (Best:0.6080), test acc: 0.6040 (Best:0.6040)
In epoch 25, loss: 0.8792, val acc: 0.6820 (Best:0.6820), test acc: 0.6670 (Best:0.6670)
In epoch 30, loss: 0.5754, val acc: 0.7320 (Best:0.7320), test acc: 0.7200 (Best:0.7200)
In epoch 35, loss: 0.3410, val acc: 0.7580 (Best:0.7580), test acc: 0.7340 (Best:0.7340)
In epoch 40, loss: 0.1916, val acc: 0.7720 (Best:0.7720), test acc: 0.7570 (Best:0.7520)
In epoch 45, loss: 0.1080, val acc: 0.7740 (Best:0.7740), test acc: 0.7650 (Best:0.7590)
In epoch 50, loss: 0.0636, val acc: 0.7700 (Best:0.7740), test acc: 0.7700 (Best:0.7590)
In epoch 55, loss: 0.04